In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler

import CarDetectorFunc as cd

In [2]:
config = cd.Config()
config.use_svd = False
config.show()

color space   : YCrCb
--- spatial-bining-color ---
use svd       : False
svd K         : 75
spatial size  : (32, 32)
--- color-histogram ----
hist bins     : 64
hist range    : (0, 256)
--- HOG ---
orient        : 7
pix per cell  : 8
cell per block: 2
hog channel   : ALL
--- Car Detection ---
ROI ystart    : 400
ROI ystop     : 700
window scale  : [1, 1.5, 2, 2.5]


## Load Data

In [3]:
# images are divided up into vehicles and non-vehicles
import fnmatch
import os

images = []
# for root, dirnames, filenames in os.walk('../dataset-big'):
#     for filename in fnmatch.filter(filenames, '*.png'):
for root, dirnames, filenames in os.walk('../dataset-small'):
    for filename in fnmatch.filter(filenames, '*.jpeg'):
        images.append(os.path.join(root, filename))
# images

cars = []
notcars = []

for image in images:
    if 'non-vehicles' in image:
        notcars.append(image)
    else:
        cars.append(image)
# cars

In [4]:
cd.print_data_info(car_list=cars, notcar_list=notcars)

cars: 1196, non-cars: 1125, total: 2321
image shape: (64, 64, 3), data type: uint8


## Get Features

In [5]:
if config.use_svd:
    svd_subspace = cd.get_main_feature_vectors(cars, config)
else:
    svd_subspace = None

In [6]:
%%time
car_features = cd.extract_features(cars, config, svd_subspace)
notcar_features = cd.extract_features(notcars, config, svd_subspace)

spatial_features.shape: (3072,)
hist_features.shape: (192,)
hog_features.shape: (4116,)
spatial_features.shape: (3072,)
hist_features.shape: (192,)
hog_features.shape: (4116,)
CPU times: user 9.02 s, sys: 118 ms, total: 9.13 s
Wall time: 9.18 s


## Classifier

In [7]:
# Create an array stack of feature vectors
X = np.vstack((car_features, notcar_features)).astype(np.float64)
print('X.shape: {}'.format(X.shape))
# Define the labels vector
y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))
print('y.shape: {}'.format(y.shape))
# Fit a per-column scaler
X_scaler = StandardScaler().fit(X)
# Apply the scaler to X
scaled_X = X_scaler.transform(X)

X.shape: (2321, 7380)
y.shape: (2321,)
